In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

In [ ]:
!sudo apt-get install python3-pycoral

In [18]:
import tflite_runtime.interpreter as tflite
import os
import time
import subprocess
import pandas as pd

model_path='/content/efficientdet_lite1_384_ptq.tflite'
interpreter = tflite.Interpreter(model_path=model_path)
all_layers_details = interpreter.get_tensor_details()
all_layers_details = sorted(all_layers_details, key=lambda d: d['index'], reverse=False) 
layer_dict = {}
flag = True
for layer in all_layers_details:
    layer_name = layer['name']
    print(layer['index'], layer_name)
    proc = subprocess.Popen(f""" edgetpu_compiler   -i "{layer_name}"  "{model_path}" """, shell=True, stdout=subprocess.PIPE, )
    output = proc.communicate()[0].decode('utf-8')
    # print(output)
    for line in output.split('\n'):
       if 'Number of operations that will run on Edge TPU' in line:
         layer_dict[layer_name] = int(line.split(':')[1].strip())
         print(line)


0 serving_default_images:0
1 fpn_cells/cell_3/fnode0/resample_1_4_5/ResizeNearestNeighbor/size
Number of operations that will run on Edge TPU: 86
2 fpn_cells/cell_3/fnode1/resample_1_5_6/ResizeNearestNeighbor/size
Number of operations that will run on Edge TPU: 91
3 fpn_cells/cell_3/fnode2/resample_1_6_7/ResizeNearestNeighbor/size
Number of operations that will run on Edge TPU: 96
4 fpn_cells/cell_3/fnode3/resample_1_7_8/ResizeNearestNeighbor/size
Number of operations that will run on Edge TPU: 101
5 Reshape_8/shape
Number of operations that will run on Edge TPU: 312
6 Reshape_9/shape
Number of operations that will run on Edge TPU: 309
7 stack
Number of operations that will run on Edge TPU: 315
8 efficientnet-lite1/stem/conv2d/Conv2D
Number of operations that will run on Edge TPU: 1
9 efficientnet-lite1/stem/tpu_batch_normalization/FusedBatchNormV3
Number of operations that will run on Edge TPU: 1
10 efficientnet-lite1/blocks_0/tpu_batch_normalization/FusedBatchNormV3;efficientnet-lite

In [22]:
max_ops = layer_dict[max(layer_dict, key=layer_dict.get)]

In [26]:
layers = []
for key, value in layer_dict.items():
    if value == max_ops:
        layers.append(key)

print(layers)

for layer in layers:
    print(layer)
    proc = subprocess.Popen(f""" edgetpu_compiler  -s -i "{layer}"  "{model_path}" """, shell=True, stdout=subprocess.PIPE, )
    output = proc.communicate()[0].decode('utf-8')
    print(output)

['stack', 'class_net/class-predict/BiasAdd;class_net/class-predict/separable_conv2d_4;class_net/class-predict/separable_conv2d;class_net/class-predict/bias', 'class_net/class-predict/BiasAdd_1;class_net/class-predict/separable_conv2d_4;class_net/class-predict/separable_conv2d_1;class_net/class-predict/bias', 'Reshape', 'Reshape_2', 'Reshape_4', 'Reshape_6', 'Reshape_8', 'concat', 'Sigmoid', 'Sigmoid1', 'concat_1', 'tfl.dequantize']
stack
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 5034 ms.

Input model: /content/efficientdet_lite1_384_ptq.tflite
Input size: 5.79MiB
Output model: efficientdet_lite1_384_ptq_edgetpu.tflite
Output size: 7.64MiB
On-chip memory used for caching model parameters: 6.29MiB
On-chip memory remaining for caching model parameters: 1.21MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 322
Operation log: ef

In [27]:
!edgetpu_compiler -s -a "/content/efficientdet_lite1_384_ptq.tflite"

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 5150 ms.

Input model: /content/efficientdet_lite1_384_ptq.tflite
Input size: 5.79MiB
Output model: efficientdet_lite1_384_ptq_edgetpu.tflite
Output size: 7.64MiB
On-chip memory used for caching model parameters: 6.29MiB
On-chip memory remaining for caching model parameters: 1.21MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 322
Operation log: efficientdet_lite1_384_ptq_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 315
Number of operations that will run on CPU: 7

Operator          